In [1]:
from langgraph.graph import StateGraph , START  , END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict , Annotated
from dotenv  import load_dotenv
from pydantic import BaseModel , Field
import operator
load_dotenv()


True

In [2]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")


In [3]:
class EvaluationSchema(BaseModel):
    feedback : str = Field(description='Detailed feedback for the essay')
    score : int=  Field(description='Score out of 10 ', ge=0,le=10)
    

In [4]:
structured_model = model.with_structured_output(EvaluationSchema)
essay = """
India plays an increasingly significant role in global affairs as a rising economic and political power. Its growing economy, large population, and strategic partnerships position it as a key player in addressing global challenges and shaping the international order. India is a strong advocate for multilateralism, peace, and sustainable development, actively participating in international forums and contributing to global initiatives. 
Key Aspects of India's Role:
Economic Powerhouse:
India is one of the fastest-growing major economies in the world, with a large and diverse market, making it an attractive destination for foreign investment and a key player in global trade. 
Political Influence:
As a large democracy and a nuclear-armed state, India's political influence is growing, particularly in the Indo-Pacific region and on the global stage. 
Regional Power:

"""

In [5]:
#prompt = f"Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 { essay}"
#structured_model.invoke(prompt)

In [6]:
class UPSCState(TypedDict): 
    #input
    essay : str
    #calcualted using input
    language_feedback : str
    analysis_feedback : str
    clarity_feedback : str
    individual_score  : Annotated[list[int] , operator.add]
    average_score : float

In [ ]:
def eval_language(state : UPSCState) : 
    prompt = f"Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 {state['essay']}"
    output = structured_model.invoke(prompt)
    return {'language_feedback' :[output.feedback] , 'individual_score' : [output.score]}

def eval_analysis(state : UPSCState):
    prompt = f"Evaluate the depth of analysis  of the following essay and provide a feedback and assign a score out of 10 {state['essay']}"
    output = structured_model.invoke(prompt)
    return {'analysis_feedback' :[output.feedback] , 'individual_score' : [output.score]}

def eval_thought(state : UPSCState):
    prompt = f"Evaluate the clarity of thought  of the following essay and provide a feedback and assign a score out of 10 {state['essay']}"
    output = structured_model.invoke(prompt)
    return {'clarity_feedback' :[output.feedback] , 'individual_score' : [output.score]}

def final_eval(state : UPSCState) : 
    # summary , average

    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content

    # avg calculate
    avg_score = sum(state['individual_score'])/len(state['individual_score'])

    return {'overall_feedback': overall_feedback, 'average_score': avg_score}
    

In [8]:
# graphs nodes
graph = StateGraph(UPSCState)

graph.add_node('eval_language' , eval_language)
graph.add_node('eval_analysis' ,eval_analysis)
graph.add_node('eval_thought' , eval_thought)
graph.add_node('final_eval' ,final_eval)


#edges
graph.add_edge(START , 'eval_language')
graph.add_edge(START , 'eval_analysis')
graph.add_edge(START , 'eval_thought')

graph.add_edge('eval_language' , 'final_eval')
graph.add_edge('eval_analysis' , 'final_eval')
graph.add_edge('eval_thought', 'final_eval')

graph.add_edge('final_eval' ,END)

workflow = graph.compile()

In [9]:
initial_state = {'essay' : essay}
final_state = workflow.invoke(initial_state)
final_state

{'essay': "\nIndia plays an increasingly significant role in global affairs as a rising economic and political power. Its growing economy, large population, and strategic partnerships position it as a key player in addressing global challenges and shaping the international order. India is a strong advocate for multilateralism, peace, and sustainable development, actively participating in international forums and contributing to global initiatives. \nKey Aspects of India's Role:\nEconomic Powerhouse:\nIndia is one of the fastest-growing major economies in the world, with a large and diverse market, making it an attractive destination for foreign investment and a key player in global trade. \nPolitical Influence:\nAs a large democracy and a nuclear-armed state, India's political influence is growing, particularly in the Indo-Pacific region and on the global stage. \nRegional Power:\n\n",
 'language_feedback': ["The essay provides a good overview of India's growing role in global affairs.